In [1]:
from mnist import MNIST
from torch import nn
import torch
from torchvision import datasets, transforms
import numpy

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.4.0  Device: cpu


#### MNIST Data

In [2]:
transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,), (0.5,)),
                        ])

train_set = datasets.MNIST('./Dataset-3', download=True, train=True, transform=transform)
val_set = datasets.MNIST('./Dataset-3', download=True, train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32, shuffle=True)

## 1. Multilayer Perceptron

In [3]:
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size))

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()

print(model)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
)


In [4]:
def train(epoch, log_interval=200):
    # Set model to training mode
    model.train()
    
    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data = data.view(data.shape[0], -1)
        # Copy data to GPU if needed
        data = data.to(device)
        target = target.to(device)

        # Zero gradient buffers
        optimizer.zero_grad() 
        
        # Pass data through the network
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)

        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

In [5]:
def validate(loss_vector, accuracy_vector):
    pred_arr = []
    model.eval()
    val_loss, correct = 0, 0
    for data, target in val_loader:
        data = data.view(data.shape[0], -1)
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        for i in numpy.array(pred):
            pred_arr.append(int(i))
        correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(val_loader)
    loss_vector.append(val_loss)

    accuracy = 100. * correct.to(torch.float32) / len(val_loader.dataset)
    accuracy_vector.append(accuracy)
    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(val_loader.dataset), accuracy))
    
    return pred_arr

In [6]:
%%time
epochs = 10

lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(epoch)
    output = validate(lossv, accv)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309464
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.907003
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.376824
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.276851
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.345412
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.304346
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.207404
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.338337
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.520478
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.228887

Validation set: Average loss: 0.2728, Accuracy: 9211/10000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.250406
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.558774
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.287441
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.353965
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.112871
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.270819
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.485234
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.133416
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.11

In [7]:
# print(output)

## 2. Convolutional Neural Network

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [9]:
transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,), (0.5,)),
                        ])

train_set = datasets.MNIST('./Dataset-3', download=True, train=True, transform=transform)
val_set = datasets.MNIST('./Dataset-3', download=True, train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32, shuffle=True)

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [11]:
def train(epoch, log_interval=200):
    # Set model to training mode
    model.train()
    
    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(train_loader):
        
#         data = data.view(data.shape[0], -1)
        # Copy data to GPU if needed
        data = data.to(device)
        target = target.to(device)

        # Zero gradient buffers
        optimizer.zero_grad() 
        
        # Pass data through the network
        output = model(data)

        # Calculate loss
        loss = F.nll_loss(output, target)
        
        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

In [12]:
def validate(loss_vector, accuracy_vector):
    pred_arr = []
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for data, target in val_loader:
    #         data = data.view(data.shape[0], -1)
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            val_loss += F.nll_loss(output, target, reduction='sum').data.item()
            pred = output.data.max(1)[1] # get the index of the max log-probability
            for i in numpy.array(pred):
                pred_arr.append(int(i))
            correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(val_loader)
    loss_vector.append(val_loss)

    accuracy = 100. * correct.to(torch.float32) / len(val_loader.dataset)
    accuracy_vector.append(accuracy)
    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(val_loader.dataset), accuracy))
    
    return pred_arr

In [13]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=1, gamma=0.5)

In [14]:
%%time
epochs = 10

lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(epoch)
    output = validate(lossv, accv)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.316251
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.774800
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.935153
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.823217
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.536507
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.756309
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.388988
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.665120
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.307832
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.309558

Validation set: Average loss: 9.0548, Accuracy: 9218/10000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.373117
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.418231
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.453074
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.209779
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.334920
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.189587
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.460681
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.369078
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.39

In [15]:
# print(output)

## Support Vector Machine

In [16]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [17]:
transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,), (0.5,)),
                        ])

train_set = datasets.MNIST('./Dataset-3', download=True, train=True, transform=transform)
val_set = datasets.MNIST('./Dataset-3', download=True, train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set)
val_loader = torch.utils.data.DataLoader(val_set)

In [18]:
x_train = []
y_train = []

x_val = []
y_val = []

In [19]:
for x,y in train_loader:
    img = numpy.array(numpy.reshape(x, (28, 28))).flatten()
    x_train.append(img)
    y_train.append(int(y))
    
for x,y in val_loader:
    img = numpy.array(numpy.reshape(x, (28, 28))).flatten()
    x_val.append(img)
    y_val.append(int(y))

In [20]:
x_train = numpy.array(x_train)
y_train = numpy.array(y_train)

x_val = numpy.array(x_val)
y_val = numpy.array(y_val)

In [21]:
# linear model

model_linear = SVC(C= 0.85, kernel='poly')
model_linear.fit(x_train, y_train)

# predict
y_pred = model_linear.predict(x_val)

In [22]:
# print(y_pred)

In [23]:
# accuracy
print("accuracy:", metrics.accuracy_score(y_true=y_val, y_pred=y_pred), "\n")

# confusion matrix
print(metrics.confusion_matrix(y_true=y_val, y_pred=y_pred))

accuracy: 0.9827 

[[ 973    0    2    0    0    2    0    1    2    0]
 [   0 1131    1    1    0    1    0    1    0    0]
 [   5    1 1014    0    1    0    1    6    4    0]
 [   0    0    2  994    0    2    0    4    5    3]
 [   0    0    4    0  967    0    2    0    0    9]
 [   3    0    0    7    1  874    3    0    2    2]
 [   5    2    0    0    2    5  942    0    2    0]
 [   1    7    8    3    2    0    0 1000    0    7]
 [   4    0    2    3    3    2    2    3  951    4]
 [   2    2    0    5   10    3    0    5    1  981]]
